In [1]:
import pandas as pd

df_netflix = pd.read_parquet("netflix_limpio.parquet")

df_amazon = pd.read_parquet("amazon_limpio.parquet")

df_disney = pd.read_parquet("disney_limpio.parquet")


In [2]:
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,year_added
0,s1,movie,Dick Johnson Is Dead,Kirsten Johnson,Desconocido,united states,2021-09-25,2020,pg-13,90 min,documentaries,"As her father nears the end of his life, filmm...",9,2021
1,s2,tv show,Blood & Water,Desconocido,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",south africa,2021-09-24,2021,tv-ma,2 Seasons,"international tv shows, tv dramas, tv mysteries","After crossing paths at a party, a Cape Town t...",9,2021
2,s3,tv show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",desconocido,2021-09-24,2021,tv-ma,1 Season,"crime tv shows, international tv shows, tv act...",To protect his family from a powerful drug lor...,9,2021
3,s4,tv show,Jailbirds New Orleans,Desconocido,Desconocido,desconocido,2021-09-24,2021,tv-ma,1 Season,"docuseries, reality tv","Feuds, flirtations and toilet talk go down amo...",9,2021
4,s5,tv show,Kota Factory,Desconocido,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",india,2021-09-24,2021,tv-ma,2 Seasons,"international tv shows, romantic tv shows, tv ...",In a city of coaching centers known to train I...,9,2021
